In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [7]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi14QWLt CT3BlbkFJ6nuNH0hRFwAy9HhFfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Uncertainty-Routed CoT Prompting

Uncertainty-Routed CoT Prompting, introduced by Google (2023), is an innovative method that enhances reasoning in language models (LLMs) by strategically managing multiple reasoning paths.

## How Does Uncertainty-Routed CoT Prompting Work?

Imagine you're solving a complex problem. Instead of committing to your first approach, you might try several different solutions, then go with the one that multiple methods converge on—unless there's significant disagreement, in which case you'd take extra care with a single thorough approach.

Uncertainty-Routed CoT Prompting follows this logic in three key steps:

1. **Multiple Reasoning Paths**: The LLM generates several different chains of thought (CoT) for the same problem.

2. **Majority Analysis**: The model examines all solutions and identifies whether a majority of paths lead to the same answer. The level of agreement across paths determines the confidence level.

3. **Threshold-Based Selection**: If confidence exceeds a predetermined threshold (calculated from validation data), the majority answer is selected. Otherwise, the model generates a single "greedy" reasoning path and uses that answer instead.

This approach cleverly balances between consensus-based decision making when confidence is high and focused single-path reasoning when uncertainty prevails, leading to improved performance on complex reasoning tasks like MMLU.

In [9]:
uncertainty_routed_cot_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate solutions using Uncertainty-Routed CoT Prompting for a given user question.

    Uncertainty-Routed CoT Prompting involves generating multiple Chain-of-Thought (CoT) reasoning paths, then selecting the majority answer if it's above a certain confidence threshold. If not, it selects the answer from a single, greedily generated CoT path.

    Please structure your response in the following format:

    Step 1: Generate Multiple CoT Reasoning Paths
    [Generate 3-5 different CoT reasoning paths for the given question.]

    Step 2: Analyze Uncertainty and Majority Answer
    [Identify the majority answer among the generated paths and assess the level of agreement.]

    Step 3: Apply Threshold and Select Answer
    [If the majority answer meets the confidence threshold, select it. Otherwise, generate a single, greedy CoT path and select its answer.]

    Step 4: Final Solution
    [Provide the selected answer with a brief explanation of the reasoning process.]

    Example:
    Question: "If a train travels at 60 km/h and covers 300 km, how long does it take?"

    Step 1: Generate Multiple CoT Reasoning Paths
    Path 1:
    Distance = 300 km, Speed = 60 km/h.
    Time = Distance / Speed = 300 / 60 = 5 hours.
    Answer: 5 hours.

    Path 2:
    To find the time, divide the distance by the speed.
    300 km / 60 km/h = 5 hours.
    Answer: 5 hours.

    Path 3:
    The train travels 60 km in 1 hour.
    To cover 300 km, it takes 300 / 60 = 5 hours.
    Answer: 5 hours.

    Step 2: Analyze Uncertainty and Majority Answer
    All paths agree on the answer: 5 hours.
    Confidence is high.

    Step 3: Apply Threshold and Select Answer
    Since all paths agree, the majority answer is selected.

    Step 4: Final Solution
    The train takes 5 hours to cover 300 km.
    """),
    ("human", "{question}"),
    ("assistant", """
    I will use Uncertainty-Routed CoT Prompting to solve this problem.

    Step 1: Generate Multiple CoT Reasoning Paths
    [Generate multiple CoT reasoning paths for the question.]

    Step 2: Analyze Uncertainty and Majority Answer
    [Analyze the generated paths to identify the majority answer and assess confidence.]

    Step 3: Apply Threshold and Select Answer
    [Apply the threshold to select the answer.]

    Step 4: Final Solution
    [Provide the final solution.]
    """)
])

# Suitable Questions for Uncertainty-Routed CoT Prompting

For Uncertainty-Routed CoT Prompting, the most suitable questions are those that benefit from exploring multiple reasoning paths to manage uncertainty effectively. Here are types of questions that would be particularly well-suited to this approach:

### Complex Reasoning Problems
- Multi-step logical puzzles with potential for divergent approaches
- Probability questions with subtle considerations
- Problems with potential calculation traps or common misunderstandings
- Scenarios where intuition might lead to incorrect first impressions

### Ambiguous Analysis Scenarios
- Word problems with potentially misleading wording
- Questions where multiple valid interpretations exist
- Problems where experts might disagree on methodology
- Cases requiring weighing of competing factors

### Problems with Verification Challenges
- Questions where common errors are easy to make but hard to detect
- Mathematical problems where computational accuracy matters
- Reasoning tasks where small mistakes compound significantly
- Scenarios where different approaches produce slightly different answers

### Concrete Examples of Questions
1. "If a factory produces widgets at a rate of 800 per day with 5% defective, and each widget costs $3 to make and sells for $8, what is the expected daily profit after accounting for defects?"
   - Multiple paths might handle the defect calculation differently, revealing uncertainties

2. "A dice is rolled 5 times. What is the probability of getting exactly 3 sixes?"
   - Different approaches (combinatorial, direct probability) might yield slightly different results due to rounding or calculation errors

3. "Based on these symptoms and test results, what is the most likely diagnosis from these five possibilities?"
   - Different diagnostic reasoning paths might emphasize different factors, revealing uncertainty in medical reasoning

4. "In what year will Country A's GDP exceed Country B's GDP, given their current values and growth rates?"
   - Different modeling assumptions could lead to different projected years

5. "Is this algorithm's time complexity O(n log n) or O(n²)?"
   - Multiple analysis approaches might yield different conclusions about edge cases

Uncertainty-Routed CoT Prompting is particularly effective for these types of questions because:
- It generates multiple reasoning paths to explore different angles
- It identifies when there's strong consensus across reasoning attempts
- It detects uncertainty when different paths lead to different conclusions
- It automatically adjusts the approach based on confidence in the majority answer
- It provides a fallback strategy (greedy path) when uncertainty is high

This approach is especially powerful when dealing with problems where reasoning errors are common or where the confidence in an answer is as important as the answer itself. Unlike approaches that always pick the majority answer, this method intelligently adapts based on the level of uncertainty detected across multiple reasoning attempts.

In [10]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the Uncertainty-Routed CoT prompt and extract the final answer
def generate_uncertainty_routed_cot_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = uncertainty_routed_cot_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final solution from Step 4
    final_solution_pattern = r'Step 4: Final Solution\s*([\s\S]*?)(?=$|Step \d:)'
    final_solution_match = re.search(final_solution_pattern, full_response)
    
    if final_solution_match:
        final_solution = final_solution_match.group(1).strip()
    else:
        # Alternative pattern for different formatting
        alternative_pattern = r'Step 4:([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('uncertainty_routed_cot_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL SOLUTION ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final solution in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_uncertainty_routed_cot", generate_uncertainty_routed_cot_node)

# Graph configuration
graph_builder.set_entry_point("generate_uncertainty_routed_cot")
graph_builder.add_edge("generate_uncertainty_routed_cot", END)
graph = graph_builder.compile()

# Usage example
inputs = {
    "messages": [HumanMessage(content="If a factory produces widgets at a rate of 800 per day with 5% defective, and each widget costs $3 to make and sells for $8, what is the expected daily profit after accounting for defects?")],
    "question": None,
    "full_response": None
}

# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL SOLUTION:")

# Graph execution - only display the final solution
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_uncertainty_routed_cot":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'uncertainty_routed_cot_result.txt')")

QUESTION:
If a factory produces widgets at a rate of 800 per day with 5% defective, and each widget costs $3 to make and sells for $8, what is the expected daily profit after accounting for defects?

FINAL SOLUTION:
The expected daily profit after accounting for defects is $3680. This was determined by calculating the total revenue from non-defective widgets and subtracting the total production cost.

(Complete results saved in 'uncertainty_routed_cot_result.txt')
